# Homework 2

Please read the instructions before starting.

- Only use array manipulation functions from ```numpy```.
- You can use ```PIL``` for reading images and ```ipywidgets``` and ```display``` to display them.
- Use ```numpy``` operations and arrays as much as possible for performance criteria. Try to avoid using for-loops as they will drastically slow down your implementations for large-scale images. Slow implementations will have a penalty during grading.
- You can overwrite the template as long as the above conditions are not violated and the functionality is kept the same.

 Fill the the marked areas in the cells for each question.

## Question 1


Similar to the last question in the homework 1, implement a local filtering function using ```numpy``` and run mean and Gaussian filters of varying kernel sizes to the input image. (Note that you can use your previous implementation as a starting point)


In [3]:
from typing import List, Tuple, Any
import numpy as np
from PIL.Image import Image as ImageType
from PIL import Image

from utils import array_to_image, image_to_array
from renderer import noise_renderers


def apply_filter(image: ImageType, kernel: np.ndarray, padding: List[List[int]]) -> np.ndarray:
    """ Apply a filter with the given kernel to the zero padded input image.
        **Note:** Kernels can be rectangular.
        **Note:** You can use ```np.meshgrid``` and indexing to avoid using loops (bonus +5) for convolving.
        **Do not** use ```np.convolve``` in this question.
        **Do not** use ```np.pad```. Use index assignment and slicing with numpy and do not loop
            over the pixels for padding.

    Args:
        image (ImageType): 2D Input image
        kernel np.ndarray: 2D kernel array of odd edge sizes
        padding: List[list[int]]: List of zero paddings. Example: [[3, 2], [1, 4]]. The first list
            [3, 2] determines the padding for the width of the image while [1, 4] determines the
            padding to apply to top and bottom of the image. The resulting image will have a shape
            of ((1 + H + 4), (3 + W + 2)).

    Raises:
        ValueError: If the length of kernel edges are not odd

    Returns:
        np.ndarray: Filtered array (May contain negative values)
    """

    k_height = kernel.shape[0]
    k_width = kernel.shape[1]
    if (k_height%2 == 0 or k_width%2 == 0):
        raise ValueError('A very specific bad thing happened.')
    image_array = image_to_array(image)
    convolved_array = image_array
    
    for i in range (image_array.shape[0]):
        for j in range(image_array.shape[1]):
            mat = padding[i:i+k_height, j:j+k_width]
            # print(np.sum(np.multiply(mat, kernel)))
            convolved_array[i, j] = np.sum(np.multiply(mat, kernel))

    # print(convolved_array)
    return convolved_array
    raise NotImplementedError


def box_filter(image: ImageType, kernel_size: Tuple[int]) -> ImageType:
    """ Apply Box filter.

    Args:
        image (ImageType): 2D Input image of shape (H, W)
        kernel_size (Tuple[int]): 2D kernel size of kernel (height, width)

    Returns:
        ImageType: Filtered Image
    """
    image_array = image_to_array(image)

    A = np.zeros((int((kernel_size[0]-1)/2),image_array.shape[1]+kernel_size[1]-1))
    B = np.zeros((image_array.shape[0],int((kernel_size[1]-1)/2)))
    padded_list = np.block([[A], [B,image_array,B], [A]])

    kernel = np.ones((kernel_size[0], kernel_size[1]))/(kernel_size[0]*kernel_size[1])
 
    result = apply_filter(image, kernel, padded_list)
    return array_to_image(result)
    raise NotImplementedError


def gaussian_filter(image: ImageType, kernel_size: Tuple[int], sigma: float) -> ImageType:
    """ Apply Gauss filter that is centered and has the shared standard deviation ```sigma```
    **Note:** Remember to normalize kernel before applying.
    **Note:** You can use ```np.meshgrid``` (once again) to generate Gaussian kernels

    Args:
        image (ImageType): 2D Input image of shape (H, W)
        kernel_size (Tuple[int]): 2D kernel size
        sigma (float): Standard deviation

    Returns:
        ImageType: Filtered Image
    """
    image_array = image_to_array(image)

    A = np.zeros((int((kernel_size[0]-1)/2),image_array.shape[1]+kernel_size[1]-1))
    B = np.zeros((image_array.shape[0],int((kernel_size[1]-1)/2)))
    padded_list = np.block([[A], [B,image_array,B], [A]])
    sigma = 1
    kernel = np.ones((kernel_size[0],kernel_size[1]))/(kernel_size[0]*kernel_size[1])
    for i in range (kernel.shape[0]):
        for j in range (kernel.shape[1]):
            kernel[i,j] = 1/(2*np.pi*pow(sigma,2))*np.exp(-1*(pow(j,2)+pow(i,2))/2*pow(sigma,2))
    sum = np.sum(kernel)
    kernel = kernel/sum

    result = apply_filter(image, kernel, padded_list)
    return array_to_image(result)
    # raise NotImplementedError


In [4]:
# Test your above functions before running this cell
image = Image.open("noisy_image.png")
noise_renderers(image, gaussian_filter, box_filter)

> Discuss the differences of the box and Gaussian filters in this Markdown cell.

**Answer**: 

For the frequency domain, the gaussian filter is better than the box (mean) filter. The box filter is insufficient to separate frequencies but can be calculated faster than Gaussian blur. If you don't want to separate the frequencies box filter can be a good choice to remove noise in the image. Gaussian filters weigh pixels in a bell curve around the center pixel. This means that pixels farther away have lower weights. Box filter, average the pixel values ​​of all neighboring pixels. This is equivalent to giving equal weight to all pixels around the center, regardless of the distance from the center pixel. Near pixels have a greater effect on flattened rather than further pixels. However, in the box filter, all pixels of the kernel are given equal weight. Therefore, a better blur image is obtained with the gaussian filter.

## Question 2

Implement vertical and horizontal derivatives with 1D kernels of length 3. Use ```apply_filter``` function to do so.

**Note:** You can use kernels of shape (1, k) or (k, 1) as 1D kernels. 

In [5]:
from renderer import edge_renderers

def horizontal_derivative(image: ImageType) -> ImageType:
    """ Return the horizontal derivative image with same padding.
    **Note**: Pad the input image so that the output image has the same size/shape.

    Args:
        image (ImageType): 2D Input Image of shape (H, W)

    Returns:
        ImageType: Derivative image of shape (H, W).
    """
    kernel = np.array([[1,0,-1],])
    # print(kernel.shape)
    image_array = image_to_array(image)
    B = np.zeros((image_array.shape[0],1))
    padded_list = np.block([B,image_array,B])
    # print(padded_list)

    result = apply_filter(image, kernel, padded_list)
    return array_to_image(result)    
    raise NotImplementedError


def vertical_derivative(image: ImageType) -> ImageType:
    """ Return the vertical derivative image with same padding.
    **Note**: Pad the input image so that the output image has the same size/shape.

    Args:
        image (ImageType): 2D Input Image of shape (H, W)

    Returns:
        ImageType: Derivative image of shape (H, W).
    """
    kernel = np.array([[1],[0],[-1]])
    # print(kernel.shape)
    image_array = image_to_array(image)

    A = np.zeros((1,image_array.shape[1]))
    padded_list = np.block([[A],[image_array],[A]])
    # print(padded_list)

    result = apply_filter(image, kernel, padded_list)
    return array_to_image(result)
    raise NotImplementedError


In [6]:
# Test your above functions before running this cell
image = Image.open("building.png")
edge_renderers(
    (image, "Original Image"),
    (vertical_derivative(image), "Vertical"),
    (horizontal_derivative(image), "Horizontal"),
)

### Sobel Operator

Implement Sobel filter for edge detection using 3x3 kernels.

Combine the output of the vertical and horizontal Sobel operators, namely $S_x$ and $S_y$, to obtain gradient image.
 

In [9]:
def sobel_vertical(image: ImageType) -> np.ndarray:
    """ Return the output of the vertical Sobel operator with same padding.

    Args:
        image (ImageType): 2D Input Image of shape (H, W)

    Returns:
        np.ndarray: Derivative array of shape (H, W).
    """
    kernel = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
    image_array = image_to_array(image)

    A = np.zeros((1,image_array.shape[1]+2))
    B = np.zeros((image_array.shape[0],1))
    padded_list = np.block([[A], [B,image_array,B], [A]])

    result = apply_filter(image, kernel, padded_list)
    return result
    raise NotImplementedError


def sobel_horizontal(image: ImageType) -> np.ndarray:
    """ Return the output of the horizontal Sobel operator with same padding.

    Args:
        image (ImageType): 2D Input Image of shape (H, W)

    Returns:
        np.ndarray: Derivative array of shape (H, W).
    """
    kernel = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
    image_array = image_to_array(image)

    A = np.zeros((1,image_array.shape[1]+2))
    B = np.zeros((image_array.shape[0],1))
    padded_list = np.block([[A], [B,image_array,B], [A]])

    result = apply_filter(image, kernel, padded_list)
    return result
    # raise NotImplementedError


def gradient_image(image: ImageType) -> ImageType:
    """ Return the gradient image calculated by combining the output of Sobel filters.

    Args:
        image (ImageType): 2D Input Image of shape (H, W)

    Returns:
        ImageType: Derivative image of shape (H, W).
    """
    array_v = np.absolute(sobel_vertical(image))
    array_h = np.absolute(sobel_horizontal(image))
    return array_to_image(array_h + array_v)
    
    raise NotImplementedError


In [10]:
# Test your above functions before running this cell
image = Image.open("building.png")
edge_renderers(
    (image, "Original Image"),
    (gradient_image(image), "Edge Image"),
)

HELİN ASLI AKSOY                                                                                                      
150200705